## Exercise 1: Extended IP Analysis

The following code uses the Python `ipaddress` module to analyze an IP address provided in CIDR notation. It calculates:

- The broadcast address
- The first and last usable host addresses (for networks with at least 2 hosts)
- The number of usable hosts

You can experiment by changing the input CIDR (e.g., `/24`, `/30`) to see how the network changes.

In [6]:
import ipaddress

def analyse_ip(ip_str):
    # Create an IP interface object from a CIDR notation string (e.g., '192.168.1.1/24')
    ip_intf = ipaddress.ip_interface(ip_str)
    net = ip_intf.network
    ip_addr = ip_intf.ip
    
    print(f"Address: {ip_addr}")
    print(f"Network: {net}")
    print(f"Netmask: {ip_intf.netmask}")
    
    # Broadcast address
    print(f"Broadcast Address: {net.broadcast_address}")
    
    # Calculate usable hosts
    hosts = list(net.hosts())
    if len(hosts) >= 2:
        first_host = hosts[0]
        last_host = hosts[-1]
    else:
        first_host = last_host = None
    
    print(f"First usable host: {first_host}")
    print(f"Last usable host: {last_host}")
    
    # Number of usable hosts (for IPv4, subtract network and broadcast if applicable)
    num_usable = net.num_addresses - 2 if net.num_addresses > 2 else net.num_addresses
    print(f"Number of usable hosts: {num_usable}")
    
    print(f"Is private: {ip_addr.is_private}")
    print(f"Is global: {ip_addr.is_global}")

# Example usage
print("--- Analyzing 192.168.1.1/24 ---")
analyse_ip('192.168.1.1/24')

--- Analyzing 192.168.1.1/24 ---
Address: 192.168.1.1
Network: 192.168.1.0/24
Netmask: 255.255.255.0
Broadcast Address: 192.168.1.255
First usable host: 192.168.1.1
Last usable host: 192.168.1.254
Number of usable hosts: 254
Is private: True
Is global: False


## Exercise 2: Analyzing Your Device's IP Address

This cell obtains your computer's hostname and IP address, then calls the `analyse_ip` function to analyze it. (Note that if your computer has multiple interfaces, this might not capture all details.)

In [7]:
import socket
import ipaddress

# Get your computer's IP address
hostname = socket.gethostname()
IPAddr = socket.gethostbyname(hostname)

print(f"Your Computer Name: {hostname}")
print(f"Your Computer IP Address: {IPAddr}")

# Analyze the IP
ip = ipaddress.ip_interface(IPAddr + "/24")

print(f"Network: {ip.network}")
print(f"Netmask: {ip.netmask}")
print(f"Broadcast Address: {ip.network.broadcast_address}")
print(f"First Usable Host: {list(ip.network.hosts())[0]}")
print(f"Last Usable Host: {list(ip.network.hosts())[-1]}")
print(f"Number of Usable Hosts: {ip.network.num_addresses - 2}")
print(f"Private IP: {ip.ip.is_private}")
print(f"Global IP: {ip.ip.is_global}")


Your Computer Name: Anas-MBP.communityfibre.co.uk
Your Computer IP Address: 192.168.1.215
Network: 192.168.1.0/24
Netmask: 255.255.255.0
Broadcast Address: 192.168.1.255
First Usable Host: 192.168.1.1
Last Usable Host: 192.168.1.254
Number of Usable Hosts: 254
Private IP: True
Global IP: False


## Exercise 3: Analyzing a University Website's IP Address

Enter the hostname of your university website. The script retrieves its IP address using `socket.gethostbyname()` and analyzes it using the `analyse_ip` function.

In [8]:
university_host = input("Enter the university website hostname (e.g., www.exampleuni.edu): ")
uni_ip = socket.gethostbyname(university_host)
print(f"IP address of {university_host} is {uni_ip}")

# Analyze the university IP address with a sample CIDR (/24). Adjust as needed.
analyse_ip(uni_ip + '/24')

IP address of www.gold.ac.uk is 159.100.136.66
Address: 159.100.136.66
Network: 159.100.136.0/24
Netmask: 255.255.255.0
Broadcast Address: 159.100.136.255
First usable host: 159.100.136.1
Last usable host: 159.100.136.254
Number of usable hosts: 254
Is private: False
Is global: True


## Exercise 4 (Challenge): Subnetting Plan for a Company

A company with the network address `172.16.0.0/16` has four departments with the following host requirements:

- Engineering: 30 hosts
- Marketing: 15 hosts
- Finance: 10 hosts
- HR: 5 hosts

For each department, you need to determine a subnet that provides enough usable host addresses. (Remember: For an IPv4 subnet, the number of usable hosts is `2^(32 - prefix) - 2`.)

Below is a simple calculation to help you decide on the minimum prefix length needed for each department.

In [9]:
import math

def min_prefix_for_hosts(required_hosts):
    # Calculate the number of bits needed for hosts: 2^(32 - prefix) - 2 >= required_hosts
    # Solve for prefix: prefix <= 32 - ceil(log2(required_hosts + 2))
    bits_needed = math.ceil(math.log2(required_hosts + 2))
    prefix = 32 - bits_needed
    return prefix

departments = {
    "Engineering": 30,
    "Marketing": 15,
    "Finance": 10,
    "HR": 5
}

print("Subnetting Plan for network 172.16.0.0/16")
for dept, hosts in departments.items():
    prefix = min_prefix_for_hosts(hosts)
    usable = (2 ** (32 - prefix)) - 2
    print(f"{dept}: Requires {hosts} hosts → Minimum Prefix /{prefix} with {usable} usable hosts")

# Note: This gives you a starting point. You may choose to allocate more addresses
# for future growth or for organizational reasons.

Subnetting Plan for network 172.16.0.0/16
Engineering: Requires 30 hosts → Minimum Prefix /27 with 30 usable hosts
Marketing: Requires 15 hosts → Minimum Prefix /27 with 30 usable hosts
Finance: Requires 10 hosts → Minimum Prefix /28 with 14 usable hosts
HR: Requires 5 hosts → Minimum Prefix /29 with 6 usable hosts


## 5. DHCP Simulation

The following code simulates the basic DHCP DORA process:

1. **DHCPDISCOVER:** The client broadcasts a request for an IP address.
2. **DHCPOFFER:** The server offers an IP address from its pool.
3. **DHCPREQUEST:** The client requests the offered IP address.
4. **DHCPACK:** The server acknowledges the assignment, and the client is configured.

Run the cell below to simulate the DHCP process.

In [10]:
def send_discover(client_mac):
    print("\n[CLIENT] Step 1: Sending DHCP DISCOVER")
    return { "type": "DISCOVER", "mac": client_mac }

def make_offer(discover, ip_pool):
    print("\n[SERVER] Step 2: Making DHCP OFFER")
    if not ip_pool:
        print("No IPs available!")
        return None
    offered_ip = ip_pool.pop(0)
    return { "type": "OFFER", "mac": discover["mac"], "ip": offered_ip }

def send_request(offer):
    print("\n[CLIENT] Step 3: Sending DHCP REQUEST")
    return { "type": "REQUEST", "mac": offer["mac"], "ip": offer["ip"] }

def send_ack(request, leases):
    print("\n[SERVER] Step 4: Sending DHCP ACK")
    leases[request["mac"]] = request["ip"]
    return { "type": "ACK", "mac": request["mac"], "ip": request["ip"] }

def dhcp_simulation():
    # Server configuration
    server = {
        "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"],
        "leases": {}
    }
    
    # Client configuration
    client = { "mac": "AA:BB:CC:DD:EE:FF", "ip": None }
    
    print("=== Simple DHCP Simulation ===")
    discover = send_discover(client["mac"])
    offer = make_offer(discover, server["ip_pool"])
    if not offer:
        return
    request = send_request(offer)
    ack = send_ack(request, server["leases"])
    client["ip"] = ack["ip"]
    
    print("\n=== Result ===")
    print(f"Client {client['mac']} got IP: {client['ip']}")
    print("Server leases:", server["leases"])

dhcp_simulation()

=== Simple DHCP Simulation ===

[CLIENT] Step 1: Sending DHCP DISCOVER

[SERVER] Step 2: Making DHCP OFFER

[CLIENT] Step 3: Sending DHCP REQUEST

[SERVER] Step 4: Sending DHCP ACK

=== Result ===
Client AA:BB:CC:DD:EE:FF got IP: 192.168.1.100
Server leases: {'AA:BB:CC:DD:EE:FF': '192.168.1.100'}
